## 0- Imports

In [ ]:
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import yaml

from ultralytics import YOLO

---

## 1- Preparing the Dataset

I use a YAML file since it is the best practice with the YOLO models. To use such file, first, I create a convenient version of the dataset

In [2]:
base_dir = os.getcwd()
annotations_dir = os.path.join(base_dir, 'Annotations')
images_dir = os.path.join(base_dir, 'Images')
imagesets_dir = os.path.join(base_dir, 'ImageSets')

In [3]:
dataset_dir = os.path.join(base_dir, 'dataset')
os.makedirs(os.path.join(dataset_dir, 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'train', 'labels'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'val', 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'val', 'labels'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'test', 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_dir, 'test', 'labels'), exist_ok=True)

In [4]:
def convert_to_yolo_format(bbox, img_width, img_height):
    x_min, y_min, x_max, y_max, class_id = bbox
    
    # Convert to center format
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    width = x_max - x_min
    height = y_max - y_min
    
    # Normalize to 0-1
    x_center /= img_width
    y_center /= img_height
    width /= img_width
    height /= img_height
    
    # YOLO uses zero-indexed classes
    class_id = int(class_id) - 1
    
    return [class_id, x_center, y_center, width, height]

In [ ]:
# Process each split (train, val, test)
for split in ['train', 'val', 'test']:
    # Read filenames from split files
    with open(os.path.join(imagesets_dir, f'{split}.txt'), 'r') as f:
        filenames = [line.strip() for line in f.readlines()]
    
    for filename in filenames:
        # Image path
        image_path = os.path.join(images_dir, f'{filename}.png')
            
        # Get image dimensions
        with Image.open(image_path) as img:
            img_width, img_height = img.size
        
        # Annotation path
        annotation_path = os.path.join(annotations_dir, f'{filename}.txt')
        
        # Destination paths
        dest_img_path = os.path.join(dataset_dir, split, 'images', os.path.basename(image_path))
        dest_label_path = os.path.join(dataset_dir, split, 'labels', f'{filename}.txt')
        
        # Copy image
        shutil.copy(image_path, dest_img_path)
        
        # Convert and save annotation
        with open(annotation_path, 'r') as f:
            bbox_lines = f.readlines()
            
        with open(dest_label_path, 'w') as f:
            for line in bbox_lines:
                bbox = list(map(float, line.strip().split()))
                yolo_bbox = convert_to_yolo_format(bbox, img_width, img_height)
                f.write(' '.join(map(str, yolo_bbox)) + '\n')

Dataset conversion complete!


In [ ]:
data_yaml = {
    'path': dataset_dir,  # Absolute path to dataset directory
    'train': os.path.join(dataset_dir, 'train', 'images'),
    'val': os.path.join(dataset_dir, 'val', 'images'),
    'test': os.path.join(dataset_dir, 'test', 'images'),
    'names': {
        0: 'car'
    }
}

# Write to file
with open('dataset.yaml', 'w') as f:
    yaml.dump(data_yaml, f, sort_keys=False)

Created dataset.yaml with path: c:\Users\Bora\Desktop\HU\AIN433\Assignment 3\dataset


---

## 2- Training

First, I define a global training function that can be used for all settings.

In [28]:
def train_yolo_frozen_blocks(blocks_to_freeze=3, optimizer='Adam', lr=0.001, batch_size=16, epochs=30):
    """
    Train YOLOv8n with specific blocks frozen and hyperparameters.
    
    Args:
        blocks_to_freeze: Number of blocks to freeze from the start
        optimizer: Optimizer to use ('Adam', 'SGD', 'AdamW')
        lr: Learning rate
        batch_size: Batch size for training
        epochs: Number of epochs
    
    Returns:
        Trained model and training results
    """
    # Load a pretrained YOLOv8n model
    model = YOLO(os.path.join(base_dir, 'yolov8n.pt'))
    
    # Freeze the first blocks_to_freeze blocks
    for i in range(blocks_to_freeze):
        for param in model.model.model[i].parameters():
            param.requires_grad = False
        print(f"Block {i} frozen")
    
    # Define training hyperparameters
    training_args = {
        'data': 'dataset.yaml',
        'epochs': epochs,
        'imgsz': 640,
        'batch': batch_size,
        'optimizer': optimizer.lower(),
        'lr0': lr,
        'name': f'freeze_{blocks_to_freeze}_opt_{optimizer}_lr_{lr}_batch_{batch_size}'
    }
    
    # Train the model
    results = model.train(**training_args)
    
    return model, results

Plotting is automatically done by YOLO trainer, making our lives much easier :)

In [64]:
def visualize_results(results):
    """
    Visualize training results and prediction comparisons from YOLOv8 output.
    
    Args:
        results: The results object returned by model.train()
    """
    # Get the save directory
    save_dir = results.save_dir
    print(f"Loading visualizations from: {save_dir}")
    
    # 1. Display the results.png which contains all the metrics and curves
    results_img_path = os.path.join(save_dir, 'results.png')
    if os.path.exists(results_img_path):
        plt.figure(figsize=(12, 8))
        img = plt.imread(results_img_path)
        plt.imshow(img)
        plt.axis('off')
        plt.title('Training Results and Metrics')
        plt.tight_layout()
        plt.show()
    else:
        print(f"Warning: Results image not found at {results_img_path}")
    
    # 2. Display validation batch images side by side (ground truth vs predictions)
    labels_img_path = os.path.join(save_dir, 'val_batch0_labels.jpg')
    preds_img_path = os.path.join(save_dir, 'val_batch0_pred.jpg')
    
    if os.path.exists(labels_img_path) and os.path.exists(preds_img_path):
        plt.figure(figsize=(18, 8))
        
        # Ground truth
        plt.subplot(1, 2, 1)
        img_labels = plt.imread(labels_img_path)
        plt.imshow(img_labels)
        plt.axis('off')
        plt.title('Ground Truth Labels')
        
        # Predictions
        plt.subplot(1, 2, 2)
        img_preds = plt.imread(preds_img_path)
        plt.imshow(img_preds)
        plt.axis('off')
        plt.title('Model Predictions')
        
        plt.tight_layout()
        plt.show()
    else:
        print(f"Warning: Validation images not found at {labels_img_path} or {preds_img_path}")

### 2.1- Freezing the First 3 Blocks

In [ ]:
# Train with first 3 blocks frozen (stem + first two backbone blocks)
model, results = train_yolo_frozen_blocks(
    blocks_to_freeze=3,
    optimizer='Adam',
    lr=0.001,
    batch_size=16,
    epochs=30
)

visualize_results(results)

Block 0 frozen
Block 1 frozen
Block 2 frozen
New https://pypi.org/project/ultralytics/8.3.126 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.49  Python-3.10.0 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 8192MiB)
engine\trainer: task=detect, mode=train, model=c:\Users\Bora\Desktop\HU\AIN433\Assignment 3\yolov8n.pt, data=dataset.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=freeze_3_opt_Adam_lr_0.001_batch_168, exist_ok=False, pretrained=True, optimizer=adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer

train: Scanning C:\Users\Bora\Desktop\HU\AIN433\Assignment 3\dataset\train\labels.cache... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]
val: Scanning C:\Users\Bora\Desktop\HU\AIN433\Assignment 3\dataset\val\labels.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]


Plotting labels to runs\detect\freeze_3_opt_Adam_lr_0.001_batch_168\labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\freeze_3_opt_Adam_lr_0.001_batch_168
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30      5.69G      1.336      1.432     0.9808       1084        640: 100%|██████████| 63/63 [00:18<00:00,  3.49it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.76it/s]

                   all        200      11906      0.963      0.876      0.957      0.659


...

                   all        200      11906      0.993      0.983      0.994      0.824



30 epochs completed in 1.024 hours.
Optimizer stripped from runs\detect\freeze_3_opt_Adam_lr_0.001_batch_168\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\freeze_3_opt_Adam_lr_0.001_batch_168\weights\best.pt, 6.2MB

Validating runs\detect\freeze_3_opt_Adam_lr_0.001_batch_168\weights\best.pt...
Ultralytics 8.3.49  Python-3.10.0 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 8192MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:16<00:00,  2.38s/it]


                   all        200      11906      0.993      0.983      0.994      0.824
Speed: 1.3ms preprocess, 7.0ms inference, 0.0ms loss, 8.6ms postprocess per image
Results saved to runs\detect\freeze_3_opt_Adam_lr_0.001_batch_168


KeyError: 'train/box_loss'

In [ ]:
model2, results2 = train_yolo_frozen_blocks(
    blocks_to_freeze=3,
    optimizer='AdamW',  
    lr=0.001,
    batch_size=16,
    epochs=30
)

visualize_results(results2)

In [ ]:
model3, results3 = train_yolo_frozen_blocks(
    blocks_to_freeze=3,
    optimizer='Adam',
    lr=0.003,  
    batch_size=32,  
    epochs=30
)

visualize_results(results3)